In [1]:
# import pandas & sqlite3
import pandas as pd
import sqlite3

In [2]:
# read sql using pandas
conn = sqlite3.connect('titanic.db')
c = conn.cursor()
passengers = pd.read_sql('SELECT * FROM passengers',conn)
passengers.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,None,S


In [3]:
# print first 10 rows
data = c.execute('SELECT * FROM passengers LIMIT 10')

print(*data.fetchall(),sep="\n")
# OR
# for row in c:
#     print(row)

c.close()

(1, 0, 3, 'Braund, Mr. Owen Harris', 'male', '22', 1, 0, 'A/5 21171', 7.25, None, 'S')
(2, 1, 1, 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)', 'female', '38', 1, 0, 'PC 17599', 71.2833, 'C85', 'C')
(3, 1, 3, 'Heikkinen, Miss. Laina', 'female', '26', 0, 0, 'STON/O2. 3101282', 7.925, None, 'S')
(4, 1, 1, 'Futrelle, Mrs. Jacques Heath (Lily May Peel)', 'female', '35', 1, 0, '113803', 53.1, 'C123', 'S')
(5, 0, 3, 'Allen, Mr. William Henry', 'male', '35', 0, 0, '373450', 8.05, None, 'S')
(6, 0, 3, 'Moran, Mr. James', 'male', None, 0, 0, '330877', 8.4583, None, 'Q')
(7, 0, 1, 'McCarthy, Mr. Timothy J', 'male', '54', 0, 0, '17463', 51.8625, 'E46', 'S')
(8, 0, 3, 'Palsson, Master. Gosta Leonard', 'male', '2', 3, 1, '349909', 21.075, None, 'S')
(9, 1, 3, 'Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)', 'female', '27', 0, 2, '347742', 11.1333, None, 'S')
(10, 1, 2, 'Nasser, Mrs. Nicholas (Adele Achem)', 'female', '14', 1, 0, '237736', 30.0708, None, 'C')


In [4]:
# pip install ipython-sql
# pip install --upgrade ipython

# Loading External SQL Module
%load_ext sql

# Connecting to Database
%sql sqlite:///titanic.db

In [5]:
# Rename old column
%%sql
ALTER TABLE passengers
RENAME COLUMN Age TO Age_text

 * sqlite:///titanic.db
Done.


1


In [6]:
# Add new column
%%sql
ALTER TABLE passengers
ADD Age INTEGER

 * sqlite:///titanic.db
Done.


[]

In [7]:
# Copy values from old to new column
%%sql
UPDATE passengers
SET Age = Age_text

 * sqlite:///titanic.db
891 rows affected.


[]

In [8]:
# Remove old column
%%sql
ALTER TABLE passengers
DROP COLUMN Age_text

 * sqlite:///titanic.db
(sqlite3.OperationalError) near "DROP": syntax error
[SQL: ALTER TABLE passengers
DROP COLUMN Age_text]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [9]:
# Verify data type of new column
%sql SELECT typeof(Age) FROM passengers LIMIT 5

 * sqlite:///titanic.db
Done.


typeof(Age)
integer
integer
integer
integer
integer


In [10]:
# Update 0 to NULL
%%sql
UPDATE passengers
SET Fare = NULL
WHERE Fare = 0

 * sqlite:///titanic.db
15 rows affected.


[]

In [11]:
# Add new column with data type TEXT
%%sql
ALTER TABLE passengers
ADD Age_Group TEXT

 * sqlite:///titanic.db
Done.


[]

In [12]:
# Categorize into age group
%%sql
UPDATE passengers
SET Age_Group = (
    CASE
        WHEN (Age < 13) THEN 'children'
        WHEN (Age >= 13) AND (Age < 19) THEN 'adolescents'
        WHEN (Age >= 19) AND (Age < 60) THEN 'adults'
        WHEN (Age >= 60) THEN 'seniors'
        ELSE NULL
    END)

 * sqlite:///titanic.db
891 rows affected.


[]

In [13]:
# Survival rate and distribution
%%sql
SELECT Survived, 
    COUNT(Survived) AS Headcount,
    round(COUNT(*) * 100.0 /(SELECT COUNT(*) FROM passengers),2) AS Percentage
FROM passengers
GROUP BY Survived

 * sqlite:///titanic.db
Done.


Survived,Headcount,Percentage
0,549,61.62
1,342,38.38


In [14]:
# Socioeconomic status
%%sql
SELECT
(SELECT COUNT(Pclass) FROM passengers WHERE Pclass = 1) AS first_class,
(SELECT COUNT(Pclass) FROM passengers WHERE Pclass = 2) AS second_class,
(SELECT COUNT(Pclass) FROM passengers WHERE Pclass = 3) AS third_class

 * sqlite:///titanic.db
Done.


first_class,second_class,third_class
216,184,491


In [15]:
# Gender
%%sql
SELECT
(SELECT COUNT(Sex) FROM passengers WHERE Sex = 'male') AS male,
(SELECT COUNT(Sex) FROM passengers WHERE Sex = 'female') AS female

 * sqlite:///titanic.db
Done.


male,female
577,314


In [16]:
# Age categorisation
%%sql
SELECT
(SELECT COUNT(Age_Group) FROM passengers WHERE Age_Group = 'children') AS 'children',
(SELECT COUNT(Age_Group) FROM passengers WHERE Age_Group = 'adolescents') AS 'adolescents',
(SELECT COUNT(Age_Group) FROM passengers WHERE Age_Group = 'adults') AS 'adults',
(SELECT COUNT(Age_Group) FROM passengers WHERE Age_Group = 'seniors') AS 'seniors'

 * sqlite:///titanic.db
Done.


children,adolescents,adults,seniors
69,70,549,26


In [17]:
# Survival status based on Socioeconomic status
%%sql
WITH table1 AS (
    SELECT Pclass, COUNT(Survived) AS survive
    FROM passengers
    WHERE Survived = 1
    GROUP BY Pclass),
table2 AS (
    SELECT Pclass, COUNT(Survived) AS didnotsurvive
    FROM passengers
    WHERE Survived = 0
    GROUP BY Pclass)
SELECT table1.Pclass, survive, didnotsurvive
FROM table1 JOIN table2
    ON table1.Pclass = table2.Pclass

 * sqlite:///titanic.db
Done.


Pclass,survive,didnotsurvive
1,136,80
2,87,97
3,119,372


In [18]:
# Survival status based on Gender
%%sql
WITH table1 AS (
    SELECT Sex, COUNT(Survived) AS survive
    FROM passengers
    WHERE Survived = 1
    GROUP BY Sex),
table2 AS (
    SELECT Sex, COUNT(Survived) AS didnotsurvive
    FROM passengers
    WHERE Survived = 0
    GROUP BY Sex)
SELECT table1.Sex, survive, didnotsurvive
FROM table1 JOIN table2
    ON table1.Sex = table2.Sex

 * sqlite:///titanic.db
Done.


Sex,survive,didnotsurvive
female,233,81
male,109,468


In [19]:
# Survival status based on Age Group
%%sql
WITH table1 AS (
    SELECT Age_Group, COUNT(Survived) AS survive
    FROM passengers
    WHERE Survived = 1 AND Age_Group IS NOT NULL
    GROUP BY Age_Group),
table2 AS (
    SELECT Age_Group, COUNT(Survived) AS didnotsurvive
    FROM passengers
    WHERE Survived = 0 AND Age_Group IS NOT NULL
    GROUP BY Age_Group)
SELECT table1.Age_Group, survive, didnotsurvive
FROM table1 JOIN table2
    ON table1.Age_Group = table2.Age_Group

 * sqlite:///titanic.db
Done.


Age_Group,survive,didnotsurvive
adolescents,30,40
adults,213,336
children,40,29
seniors,7,19


In [21]:
# Survival rate and distribution for passengers without family onboard
%%sql
SELECT Survived,
    COUNT(*) AS Headcount,
    round(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM passengers WHERE Parch = 0 AND SibSp = 0),2) AS Percentage
FROM passengers
WHERE Parch = 0 AND SibSp = 0
GROUP BY Survived

 * sqlite:///titanic.db
Done.


Survived,Headcount,Percentage
0,374,69.65
1,163,30.35


In [22]:
# Survival status based on Number of family relations onboard
%%sql
WITH table1 AS (
    SELECT Parch, SibSp, COUNT(Survived) AS survive
    FROM passengers
    WHERE Survived = 1
    GROUP BY Parch, SibSp),
table2 AS (
    SELECT Parch, SibSp, COUNT(Survived) AS didnotsurvive
    FROM passengers
    WHERE Survived = 0
    GROUP BY Parch, SibSp)
SELECT table1.Parch, table1.SibSp, survive, didnotsurvive
FROM table1 JOIN table2
    ON table1.Parch = table2.Parch AND table1.SibSp = table2.SibSp

 * sqlite:///titanic.db
Done.


Parch,SibSp,survive,didnotsurvive
0,0,163,374
0,1,64,59
0,2,4,12
1,0,25,13
1,1,34,23
1,2,6,1
2,0,21,8
2,1,12,7
2,2,2,2
2,3,2,5
